In [ ]:
#default_exp engines.icevision.models

 # Models
 
 > Utilities to customize Mask R-CNN anchor sizes, and save icevision models with their configs

In [ ]:
# export

from drone_detector.imports import *
from drone_detector.utils import *
from icevision.models.torchvision import *
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNN, MaskRCNNPredictor
import torch.nn as nn
import torch

In [ ]:
# export

def mask_rcnn_custom_anchors(num_classes:int,
                             backbone=None,
                             pretrained:bool=True,
                             sizes:tuple=((32,), (64,), (128,), (256,), (512,)),
                             aspect_ratios:tuple=(0.5, 1.0, 2.0),
                             min_size:int=800,
                             max_size:int=1333
    
    ) -> nn.Module:
    "Make icevision Mask RCNN with custom anchors. Default values are torchvision defaults"
    
    
    if backbone is None:
        backbone = mask_rcnn.backbones.resnet50_fpn(pretrained=pretrained)
    
    rpn_anchor_generator = AnchorGenerator(sizes=sizes,
                                           aspect_ratios=aspect_ratios)
    
    rpn_head = RPNHead(256, rpn_anchor_generator.num_anchors_per_location()[0])
    
    in_features = 1024
    
    box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    in_features_mask = 256
    
    hidden_layer = 256
    mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
    
    mask_rcnn_kwargs = {
        'rpn_anchor_generator': rpn_anchor_generator,
        'rpn_head': rpn_head,
        'box_predictor': box_predictor,
        'mask_predictor': mask_predictor,
        'num_classes': None,
        'image_mean': [1., 1., 1.],
        'image_std': [1., 1., 1.], # This way no need to remove model normalization
        'min_size': min_size,
        'max_size': max_size
    }
    
    custom_model = mask_rcnn.model(backbone=backbone, remove_internal_transforms=False, **mask_rcnn_kwargs)
    
    return custom_model


def faster_rcnn_custom_anchors(num_classes:int,
                               backbone=None,
                               pretrained:bool=True,
                               sizes:tuple=((32,), (64,), (128,), (256,), (512,)),
                               aspect_ratios:tuple=(0.5, 1.0, 2.0),
                               min_size:int=800, # Default value
                               max_size:int=1333 # Default value
    
    ) -> nn.Module:
    "Make icevision Faster RCNN with custom anchors. Default values are torchvision defaults"
    
    
    if backbone is None:
        backbone = mask_rcnn.backbones.resnet50_fpn(pretrained=pretrained)
    
    rpn_anchor_generator = AnchorGenerator(sizes=sizes,
                                           aspect_ratios=aspect_ratios)
    
    rpn_head = RPNHead(256, rpn_anchor_generator.num_anchors_per_location()[0])
    
    in_features = 1024
    
    box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    faster_rcnn_kwargs = {
        'rpn_anchor_generator': rpn_anchor_generator,
        'rpn_head': rpn_head,
        'box_predictor': box_predictor,
        'num_classes': None,
        'image_mean': (1.,1.,1.),
        'image_std': (1.,1.,1.), # This way no need to remove model normalization
        'min_size': min_size,
        'max_size': max_size
    }
    
    custom_model = faster_rcnn.model(backbone=backbone, **faster_rcnn_kwargs, remove_internal_transforms=False)
    
    return custom_model

In order to easily load custom model configurations, save the required information to a config json-file. The file has a format of

* `model_type`: either `'mask_rcnn'` or `'faster_rcnn'`
* `backbone_name`: name of the torchvision model used as backbone, e.g. `'resnext101_32x8d_fpn'`
* `anchor_generator_sizes`: List of the anchor sizes, e.g. `[32,64,128,256,512]` is the default
* `aspect_ratios`: List of the used aspect ratios for anchors, e.g. `[0.5, 1, 2]` is the default
* `categories`: COCO-style dict of categories the model is trained on
* `model_fname`: filename for the trained weights

Both `model.pth` and `config.json` are saved in the same folder.

In [ ]:
# export

def save_model_and_config(outdir, 
                          model,
                          backbone_name, 
                          model_type, 
                          categories, 
                          min_size:int=800,
                          max_size:int=1333):
    if os.path.exists(outdir):
        print('Output directory exists')
        return
    os.makedirs(outdir)
    
    anchor_generator_sizes = model.rpn.anchor_generator.sizes
    aspect_ratios = model.rpn.anchor_generator.aspect_ratios[0]
    backbone_name = backbone_name
    model_path = f'{outdir}/model.pth'
    torch.save(model.state_dict(), model_path)
    
    outdict = {
        'model_type': model_type,
        'backbone_name': backbone_name,
        'anchor_generator_sizes': [s[0] for s in anchor_generator_sizes],
        'aspect_ratios': list(aspect_ratios),
        'categories': categories,
        'min_size': min_size,
        'max_size': max_size
    }
    
    with open(f'{outdir}/config.json', 'w') as dest:
        json.dump(outdict, dest)
    
    return
    
def load_rcnn_from_config(path_to_conf):
    "Load model according to config"
    
    with open(f'{path_to_conf}/config.json') as conf:
        conf_dict = json.load(conf)
        
    backbone_name = conf_dict['backbone_name']
    
    anchor_generator_sizes = tuple([(s,) for s in conf_dict['anchor_generator_sizes']])
    aspect_ratios = tuple(conf_dict['aspect_ratios'])
    
    num_classes = len(conf_dict['categories']) + 1
    min_size = conf_dict['min_size']
    max_size = conf_dict['max_size']
    
    
    if conf_dict['model_type'] == 'faster_rcnn': 
        model_type = faster_rcnn
        backbone = getattr(model_type.backbones, backbone_name)(pretrained=False)
        print('Not yet implemented')
        return
        
    elif conf_dict['model_type'] == 'mask_rcnn': 
        model_type = mask_rcnn
        backbone = getattr(model_type.backbones, backbone_name)(pretrained=False)
        custom_model = mask_rcnn_custom_anchors(num_classes=num_classes, 
                                                backbone=backbone, 
                                                sizes=anchor_generator_sizes,
                                                aspect_ratios=aspect_ratios,
                                                min_size=min_size,
                                                max_size=max_size)
    
   
    device = 'cpu' if not torch.cuda.is_available() else f'cuda:{torch.cuda.current_device()}'
    state_dict = torch.load(f'{path_to_conf}/model.pth', map_location=device)
    custom_model.load_state_dict(state_dict)
    if device != 'cpu': custom_model.to(torch.device('cuda'))
        
    return custom_model